In [3]:
import numpy as np
import pandas as pd
import pylab as pl

from sklearn.linear_model import Ridge

prediction_weights = 'prediction_weights.csv' 
prediction_rf = 'prediction_rf80.csv'
prediction_ridge = 'prediction_ridge.csv'

df_weights = pd.read_csv(prediction_weights)
df_rf = pd.read_csv(prediction_rf)
df_ridge = pd.read_csv(prediction_ridge)

rf_mean = df_rf.Sales.mean()
weights_mean = df_weights.Sales.mean()
ridge_mean = df_ridge.Sales.mean()
print(rf_mean, weights_mean, ridge_mean)

mean = (rf_mean + weights_mean + ridge_mean)/3.
print(mean)
max_dev = max([abs(rf_mean-mean), abs(weights_mean-mean), abs(ridge_mean-mean)])

rf_coeff = abs(mean - rf_mean)/max_dev
weights_coeff = abs(mean - weights_mean)/max_dev
ridge_coeff = abs(mean - ridge_mean)/max_dev

sum_coeff = rf_coeff + weights_coeff + ridge_coeff

rf_coeff = 3*rf_coeff/sum_coeff
weights_coeff = 3*weights_coeff/sum_coeff
ridge_coeff = 3*ridge_coeff/sum_coeff

assert(weights_coeff + ridge_coeff + rf_coeff == 3)

df_rf.Sales = rf_coeff*df_rf.Sales
df_weights.Sales = weights_coeff*df_weights.Sales
df_ridge.Sales = ridge_coeff*df_ridge.Sales

df_stack = pd.merge(df_rf, df_weights, on='Id')
df_stack = pd.merge(df_stack, df_ridge, on='Id')

df_result = df_stack[['Id']]
df_result['Sales'] = df_stack[['Sales_x', 'Sales_y', 'Sales']].mean(axis=1)
df_result.to_csv('prediction.csv', index=False)
df_result

OSError: File b'prediction_rf80.csv' does not exist